In [1]:
#導入所需套件
import pandas as pd
import numpy as np
import os

path='Train_DataSet\\'
file_path=pd.read_excel("Train_Quality_DataSet\\quality_size.xlsx")['key']       #將file夾名稱放進file_path
right_list=list()
left_list=list()

#使用for迴圈設定完整的.xlsx路徑，讀進raw data後放入right_list及left_list
for i in file_path:
    dat_path=path+"file_"+str(i)
#     print(dat_path)
    excel_path=os.listdir(dat_path)
    for j in excel_path:
        if j=='pressure.xlsx':
            #print(dat_path+"\\"+j)
            A=pd.read_excel(dat_path+"\\"+j)
            A=A.drop(['Unnamed: 0','TimeStamp'], axis=1)
#               print(A.shape)                        
            for k in A:
                if k=='pressure_right':
                    right_list.append([A[k]])
                elif k=='pressure_left':
                    left_list.append([A[k]])
print("right_list_shape:",str(np.array(right_list).shape))
print("left_list_shape:",str(np.array(left_list).shape))

right_list_shape: (159, 1)
left_list_shape: (159, 1)


In [2]:
#導入所需套件
from keras.preprocessing import sequence

#使用pad_sequences進行data補值，並轉換list維度
cnt=0
for i in right_list:
    right_list[cnt]=sequence.pad_sequences(i,dtype='float64',maxlen=7443)          
    cnt+=1
print("right_list_shape:",str(np.array(right_list).shape))                                  
right_list = np.reshape(right_list,(-1,7443))                           #三維轉二維
print("right_list_shape:",str(np.array(right_list).shape))              

cnt=0
for i in left_list:
    left_list[cnt]=sequence.pad_sequences(i,dtype='float64',maxlen=7443)          
    cnt+=1
print("left_list_shape:",str(np.array(left_list).shape))                                  
left_list = np.reshape(left_list,(-1,7443))                           #三維轉二維
print("left_list_shape:",str(np.array(left_list).shape))      

Using TensorFlow backend.


right_list_shape: (159, 1, 7443)
right_list_shape: (159, 7443)
left_list_shape: (159, 1, 7443)
left_list_shape: (159, 7443)


In [3]:
#導入所需套件
from sklearn import preprocessing

#對data進行正規化計算
right_list=right_list/right_list.max()
print("right_list[158]:",right_list[158])

left_list=left_list/left_list.max()
print("left_list[158]:",left_list[158])

right_list[158]: [0.         0.         0.         ... 0.00929615 0.00929615 0.00929615]
left_list[158]: [0.         0.         0.         ... 0.00837989 0.00837989 0.00837989]


In [4]:
#導入所需套件
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.utils import shuffle

#讀進quality_class_type.xlsx
path='Train_Quality_DataSet\\quality_class_type.xlsx'
class_type=pd.read_excel(path)

#labels設定
num_of_samples=right_list.shape[0]
right_labels=np.zeros((num_of_samples,),dtype='int64')
left_labels=np.zeros((num_of_samples,),dtype='int64')

for i in class_type:
    cnt=0
    if i=='right_ng_s':
        for j in class_type[i]:
            if j==1:
                right_labels[cnt]=0
            cnt+=1
    elif i=='right_normal_s':
        for j in class_type[i]:
            if j==1:
                right_labels[cnt]=1
            cnt+=1
    elif i=='right_best':
        for j in class_type[i]:
            if j==1:
                right_labels[cnt]=2
            cnt+=1
    elif i=='right_normal_l':
        for j in class_type[i]:
            if j==1:
                right_labels[cnt]=3
            cnt+=1
    elif i=='right_ng_l':
        for j in class_type[i]:
            if j==1:
                right_labels[cnt]=4
            cnt+=1
            
    elif i=='left_ng_s':
        for j in class_type[i]:
            if j==1:
                left_labels[cnt]=0
            cnt+=1
    elif i=='left_normal_s':
        for j in class_type[i]:
            if j==1:
                left_labels[cnt]=1
            cnt+=1
    elif i=='left_best':
        for j in class_type[i]:
            if j==1:
                left_labels[cnt]=2
            cnt+=1
    elif i=='left_normal_l':
        for j in class_type[i]:
            if j==1:
                left_labels[cnt]=3
            cnt+=1
    elif i=='left_ng_l':
        for j in class_type[i]:
            if j==1:
                left_labels[cnt]=4
            cnt+=1
print("right_labels:",right_labels)
print("left_labels:",left_labels)

#使用shuffle對feature、labels進行亂數配對
x_right,y_right=shuffle(right_list,right_labels,random_state=2)
# x_right_train,x_right_test,y_right_train,y_right_test_=train_test_split(x_right,y_right,test_size=0.2,random_state=2)

x_left,y_left=shuffle(left_list,left_labels,random_state=2)
# x_left_train,x_left_test,y_left_train,y_left_test=train_test_split(x_left,y_left,test_size=0.2,random_state=2)

right_labels: [3 3 3 3 3 4 3 4 3 3 3 3 3 4 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 4 3 3 3 3 3 3 3 3 4 3 4 3 4 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 4 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 4 3 4 3 3 4 3 3 3 3 3 3 3 3 3 3 3
 3 3 4 4 3 3 3 3 3 3 3]
left_labels: [3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 4 3 3 3 3 3 3 3 3 3]


In [5]:
#導入所需套件
from sklearn.linear_model import LogisticRegression

#使用LinearRegression()訓練並預測
model1=LogisticRegression(solver ='lbfgs',multi_class='auto',max_iter=2000,class_weight ={3:0.1,4:0.9},C=1e7)
score1=cross_val_score(model1,x_right,y_right,cv=5)
model1.fit(x_right,y_right)
print("Cross val score:",score1)    #[0.81818182 0.9375     0.90625    0.90322581 0.93548387]
print("Average Cross val score:",score1.mean())    #0.9001282991202346

model2=LogisticRegression(solver ='lbfgs',multi_class='auto',max_iter=2000,class_weight ={2:0.2,3:0.1,4:0.7})
score2=cross_val_score(model1,x_left,y_left,cv=5)
model2.fit(x_left,y_left)
print("Cross val score:",score2)    #[0.96875    0.96875    0.96875    0.90625    0.93548387]
print("Average Cross val score:",score2.mean())   #0.9495967741935484

Cross val score: [0.81818182 0.9375     0.90625    0.90322581 0.93548387]
Average Cross val score: 0.9001282991202346


D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Cross val score: [0.96875    0.96875    0.96875    0.90625    0.93548387]
Average Cross val score: 0.9495967741935484


In [6]:
#導入所需套件
from sklearn.externals import joblib

#模型儲存
joblib.dump(model1,'right_type.pkl')
joblib.dump(model2,'left_type.pkl')

D:\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['left_type.pkl']